In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import accuracy_score, roc_auc_score, make_scorer
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, VotingClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from imblearn.metrics import geometric_mean_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
import imblearn.pipeline
from imblearn.over_sampling import SMOTE

In [ ]:
data = pd.read_csv('148141-ensembles.txt', sep='\t')
# data = data.sample(frac=0.05)
data.head()

In [ ]:
print(data.shape)
X=data.iloc[:,0:-1].to_numpy()
y=data.iloc[:,-1].to_numpy()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
def get_metrics_cv(classifier, X, y):
    a = []
    # for scoring in [accuracy_score, geometric_mean_score, roc_auc_score]:
    for scoring in [geometric_mean_score]:
        scores = cross_val_score(classifier, X, y, cv=10, scoring=make_scorer(scoring))
        a.append((scores.mean(), scores.std()))
    return a

In [ ]:
def show_conf_matrix(estimator, X, y_true):
    y_pred = estimator.predict(X)
    plt.tight_layout()
    cm = ConfusionMatrixDisplay(confusion_matrix(y_true, y_pred))
    cm.plot()

In [ ]:
((len(data[data['class'] == 1]) / len(data)) * (len(data[data['class'] == 0]) / len(data)))

In [ ]:
plt.tight_layout()
data.hist(figsize=(50, 30), bins=100)
plt.show()

In [ ]:
data.iloc[:,:-1].boxplot(figsize=(70, 10))
plt.xticks(rotation=25)
plt.show()

In [ ]:
pca = PCA(n_components=2)
X2D = pca.fit_transform(data)
print(pca.explained_variance_ratio_)
print(np.sum(pca.explained_variance_ratio_))
plt.scatter(X2D[:, 0], X2D[:, 1], c=data['class'], alpha=0.5)
plt.tight_layout()
plt.title('PCA 2D')
plt.show()

In [ ]:
# %matplotlib widget
pca = PCA(n_components=3)
X2D = pca.fit_transform(data)
print(pca.explained_variance_ratio_)
print(np.sum(pca.explained_variance_ratio_))
fig = plt.figure()
ax = fig.add_subplot(projection='3d')
ax.scatter(X2D[:, 0], X2D[:, 1], X2D[:, 2], c=data['class'], alpha=0.5)
plt.title('PCA 3D')
plt.tight_layout()
plt.show()

In [ ]:
pca = PCA(n_components=len(data.columns)-1)
_ = pca.fit_transform(data)
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('Liczba wymiarów')
plt.xticks(np.arange(0, len(pca.explained_variance_ratio_), 10.0))
plt.ylabel('Zachowana wariancja')
plt.grid()
plt.tight_layout()
plt.show()

In [ ]:
metrics = {

}

metrics['RandomForestClassifier'] = get_metrics_cv(RandomForestClassifier().fit(X, y), X, y)
print(metrics)
metrics['AdaBoostClassifier'] = get_metrics_cv(AdaBoostClassifier(base_estimator=LogisticRegression()).fit(X, y), X, y)
print(metrics)
metrics['VotingClassifier'] = get_metrics_cv(VotingClassifier(estimators=[('lr', LogisticRegression()), ('dt', DecisionTreeClassifier()), ('knn', KNeighborsClassifier())]).fit(X, y), X, y)
print(metrics)
metrics['StackingClassifier'] = get_metrics_cv(StackingClassifier(estimators=[('lr', LogisticRegression()), ('dt', DecisionTreeClassifier()), ('knn', KNeighborsClassifier())], final_estimator=LogisticRegression()).fit(X, y), X, y)
print(metrics)

In [ ]:
for clf, values in metrics.items():
    print(f'{clf}: {[(round(x[0], 4),round(x[1],4)) for x in values]}')

In [ ]:
metrics_norm = {

}

metrics_norm['RandomForestClassifier'] = get_metrics_cv(make_pipeline(StandardScaler(), RandomForestClassifier().fit(X, y)), X, y)
print(metrics_norm)
metrics_norm['AdaBoostClassifier'] = get_metrics_cv(make_pipeline(StandardScaler(), AdaBoostClassifier(base_estimator=LogisticRegression())).fit(X, y), X, y)
print(metrics_norm)
metrics_norm['VotingClassifier'] = get_metrics_cv(make_pipeline(StandardScaler(), VotingClassifier(estimators=[('lr', LogisticRegression()), ('dt', DecisionTreeClassifier()), ('knn', KNeighborsClassifier())])).fit(X, y), X, y)
print(metrics_norm)
metrics_norm['StackingClassifier'] = get_metrics_cv(make_pipeline(StandardScaler(), StackingClassifier(estimators=[('lr', LogisticRegression()), ('dt', DecisionTreeClassifier()), ('knn', KNeighborsClassifier())], final_estimator=LogisticRegression())).fit(X, y), X, y)
print(metrics_norm)

In [ ]:
for clf, values in metrics_norm.items():
    print(f'{clf}: {[(round(x[0], 4),round(x[1],4)) for x in values]}')

In [ ]:
param_grid = {
    'n_estimators': [50, 100, 150],
    'learning_rate': [0.01, 0.1, 1, 10]
}

grid_search = GridSearchCV(estimator=make_pipeline(StandardScaler(), AdaBoostClassifier()), param_grid=param_grid, scoring=make_scorer(geometric_mean_score),cv=5, n_jobs=4, verbose=2)
grid_search.fit(X, y)

print(f"Best parameters: {grid_search.best_params_}")
print(f"Best cross-validation score: {grid_search.best_score_}")

In [ ]:
# Function to perform GridSearchCV with AdaBoost and a given base estimator
def perform_grid_search(base_estimator, param_grid):
    adaboost = AdaBoostClassifier(base_estimator=base_estimator)
    grid_search = GridSearchCV(estimator=make_pipeline(StandardScaler(), adaboost), param_grid=param_grid, scoring=make_scorer(geometric_mean_score),cv=5, n_jobs=4, verbose=2)
    grid_search.fit(X_train, y_train)
    return grid_search

# Define parameter grids for different base estimators
param_grid_dt = {
    'adaboostclassifier__base_estimator__max_depth': [1, 2, 3],
    'adaboostclassifier__n_estimators': [50, 100, 150],
    'adaboostclassifier__learning_rate': [0.01, 0.1, 1]
}

param_grid_lr = {
    'adaboostclassifier__base_estimator__C': [0.01, 0.1, 1, 10],
    'adaboostclassifier__n_estimators': [50, 100, 150],
    'adaboostclassifier__learning_rate': [0.01, 0.1, 1]
}

param_grid_svm = {
    'adaboostclassifier__base_estimator__C': [0.1, 1, 10],
    'adaboostclassifier__base_estimator__kernel': ['linear', 'rbf'],
    'adaboostclassifier__n_estimators': [50, 100],
    'adaboostclassifier__learning_rate': [0.01, 0.1, 1]
}

grid_search_dt = perform_grid_search(DecisionTreeClassifier(), param_grid_dt)
print("DecisionTree Best Parameters:", grid_search_dt.best_params_)
print("DecisionTree Best Cross-Validation Score:", grid_search_dt.best_score_)

grid_search_lr = perform_grid_search(LogisticRegression(max_iter=1000), param_grid_lr)
print("LogisticRegression Best Parameters:", grid_search_lr.best_params_)
print("LogisticRegression Best Cross-Validation Score:", grid_search_lr.best_score_)

grid_search_svm = perform_grid_search(SVC(probability=True), param_grid_svm)
print("SVM Best Parameters:", grid_search_svm.best_params_)
print("SVM Best Cross-Validation Score:", grid_search_svm.best_score_)

# Evaluate the best models on the test set
best_models = {
    "DecisionTree": grid_search_dt.best_estimator_,
    "LogisticRegression": grid_search_lr.best_estimator_,
    "SVM": grid_search_svm.best_estimator_
}

for model_name, model in best_models.items():
    y_pred = model.predict(X_test)
    test_accuracy = geometric_mean_score(y_test, y_pred)
    print(f"{model_name} Test Set G-mean: {test_accuracy}")

In [ ]:
# DecisionTree Best Parameters: {'adaboostclassifier__base_estimator__max_depth': 1, 'adaboostclassifier__learning_rate': 1, 'adaboostclassifier__n_estimators': 100}
# DecisionTree Best Cross-Validation Score: 0.7963328787509274

adaBoost = imblearn.pipeline.make_pipeline(StandardScaler(), AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=1), learning_rate=1, n_estimators=100))
tree = imblearn.pipeline.make_pipeline(StandardScaler(), DecisionTreeClassifier(max_depth=1))

In [ ]:
print(get_metrics_cv(adaBoost, X, y))
print(get_metrics_cv(tree, X, y))

In [ ]:
adaBoost.fit(X_train, y_train)
print(make_scorer(geometric_mean_score)(adaBoost, X_test, y_test))
tree.fit(X_train, y_train)
print(make_scorer(geometric_mean_score)(tree, X_test, y_test))

In [ ]:
show_conf_matrix(adaBoost, X_test, y_test)

In [ ]:
show_conf_matrix(tree, X_test, y_test)